# Global averaging of data in Google Cloud

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import intake
import json
import os
import importlib

from global_annual_means import *

experiment_list = ['piControl', 'abrupt-4xCO2', '1pctCO2',\
                   'abrupt-2xCO2', 'abrupt-0p5xCO2',\
                   'historical', 'hist-GHG', 'hist-nat', 'hist-aer',\
                  'ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585',\
                  'piClim-control', 'piClim-4xCO2', 'piClim-histall']

# for some reason this function could not be loaded from a .py file,
# it may have something to do with the function loading of another file
def error_test(exp, model, member):
    with open('../Data_availability/GoogleCloud/cloud-buterrors.json', 'r') as f:
         error_data_dict = json.load(f)
    if exp in error_data_dict:
        if model in error_data_dict[exp]:
            if member in error_data_dict[exp][model]:
                return True

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
col

In [ ]:
experiment_list[2]

In [ ]:
importlib.reload(global_annual_means)

#for exp in experiment_list:
for exp in [experiment_list[2]]:
    with open('../Data_availability/GoogleCloud/cloud-'+exp+'.json', 'r') as f:
         data_dict = json.load(f)
    print(exp, data_dict)
    #for model in ['NorCPM1']:
    for model in data_dict:
        #if model not in ['EC-Earth3','EC-Earth3-Veg']:
        for member in data_dict[model]:
            if error_test(exp, model, member) == True:
                print(exp, model, member, 'skipped because it contains errors')
                continue

            filename = model + '_' + exp + '_' + str(member) + '_means.csv'
            filepath = os.path.join('../Processed_data/Global_annual_means_csv/', model, exp)
            # check if file exists already:
            if os.path.isfile(filepath + '/' + filename):
                print(exp, model, member, 'global means already exist, and will therefore not be computed')
                continue
            else:
                print(exp, model, member)
            if sepvar_test(exp, model, member) == True:
                # load each variable separately because of an error in the time coordinates
                avg_df = global_average_sepvar(exp, model, member)
            else:
                ds = ds_cloud(exp, model, member)
                avg_df = global_averaging(ds)

            # save dataframe as csv:
            if os.path.isdir(filepath) == False:
                os.makedirs(filepath)
            avg_df.to_csv(filepath + '/' + filename)


In [ ]:
avg_df

In [ ]:
ds